In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools
from scipy.stats import norm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
total_df = pd.DataFrame(columns=['Price', 'lower','pred','upper','prob_of_growth','ticker'])

for j in companies:
    #Создаём дф и временной ряд доходностей
    company_df = df.loc[df['TICKER'] == j]
    company_df['earn'] = company_df.CLOSE.pct_change()
    series = company_df[['DATE','earn']].set_index('DATE')
    
    #Делаем train и test выборки
    historic = series.earn.iloc[:int(len(series) * 0.7)].to_list()
    test = series.earn.iloc[int(len(series) * 0.7):]
    
    
    upper_conf_int = []
    lower_conf_int = []
    predictions = []
    probabilities = []
    for i in range(len(test)):
        res = statsmodels.tsa.stattools.arma_order_select_ic(historic, ic=["aic", "bic"])
        model = ARIMA(historic, order=(res.aic_min_order[0] , 0 , res.aic_min_order[1]) )
        model_results = model.fit()
#------РАСЧЁТ ВЕРОЯТНОСТИ ПОЛОЖИТЕЛЬНОЙ ДОХОДНОСТИ------------    
        #список AR коэффициентов
        ar_params = model_results.arparams
        #делаем срез с хвоста train выборки длиной равной количеству AR коэффициентов
        ar_obs = np.array( historic[-len(ar_params):][::-1] )
        # Скалярно перемножаем их , получаем сумму произведений предыдущих значений и их коэффициентов
        if len(ar_params) != 0 :
            ar_part = np.dot(ar_params , ar_obs)
        elif len(ar_params) == 0 :
            ar_part = 0
        #список MA коэффициентов
        ma_params = model_results.maparams
        #делаем срез с хвоста train выборки длиной равной количеству MA коэффициентов
        ma_obs = np.array( model_results.resid[-len(ma_params):][::-1] )
        # Скалярно перемножаем их , получаем сумму произведений предыдущих значений и их коэффициентов
        if len(ma_params) != 0 :
            ma_part = np.dot(ma_params , ma_obs)
        elif len(ma_params) == 0 :
            ma_part = 0
        #Константа
        const = model_results.params[0]
        #Mu
        mu = ar_part+ma_part+const
        #STD
        std = np.sqrt(model_results.params[-1])
        #Сама вероятость положительной доходности в будущем месяце
        probability_of_growth = norm.cdf(mu/std, loc = 0, scale = 1)
#-----------------------------------------------------------------------    
        #ПОЛУЧЕНИЕ ДИ И ПРОГНОЗА ЦЕЛЕВОЙ ПЕРЕМЕННОЙ    
        arma_forecast = model_results.get_forecast(1)
        arma_predictions_df = arma_forecast.conf_int(alpha = 0.05)
        pred = model_results.predict(start=len(historic), end=len(historic), dynamic=False)
        predictions.append(pred[0])
        probabilities.append(probability_of_growth)
        upper_conf_int.append(arma_predictions_df[0][1])
        lower_conf_int.append(arma_predictions_df[0][0])
        historic.append(test[i])
    probabilities = pd.DataFrame(probabilities).set_index(test.index)    
    predictions = pd.DataFrame(predictions).set_index(test.index)
    upper_conf_int = pd.DataFrame(upper_conf_int).set_index(test.index)   
    lower_conf_int = pd.DataFrame(lower_conf_int).set_index(test.index)          
    #ФОРМИРОВАНИЕ ИТОГОВОГО DF 
    price_pred = pd.concat([test ,lower_conf_int, predictions , upper_conf_int , probabilities ], axis=1)
    price_pred.dropna(inplace=True)
    price_pred.columns = ['Price', 'lower','pred','upper','prob_of_growth']
    price_pred['ticker'] = j
    total_df = pd.concat([total_df,price_pred])
    

In [ ]:
total_df

In [2]:
import pandas as pd

In [8]:
full_df = pd.DataFrame(columns=['fr', 'lower','pred','upper','prob_of_growth','ticker'])


In [9]:
full_df.to_csv('frfr.csv')